In [139]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta, time

In [140]:
df1 = pd.read_excel(r"C:\Users\hussi\Downloads\Schedule-Export 2024-05-20 to 2024-05-26.xlsx")
df2 = pd.read_excel(r"C:\Users\hussi\Downloads\Schedule-Export 2024-05-13 to 2024-05-19.xlsx")



In [141]:
df1.shape

(1270, 14)

In [142]:
# Convert Date column to datetime

df1['Date'] = pd.to_datetime(df1['Date'])

# Convert Start and End times to datetime.time
df1['Start'] = pd.to_datetime(df1['Start'], format='%I:%M%p').dt.time
df1['End'] = pd.to_datetime(df1['End'], format='%I:%M%p').dt.time

In [143]:
#filter df1 where date is 05/26/2024
df1 = df1


In [144]:
df1

,Date,Start,End,Shift title,Job,Draft,Users,Location,Unpaid Breaks,Paid Breaks,Note,Note has attachments,Last status,Tasks
0,2024-05-20,01:00:00,08:00:00,NaN,Ludlow => Concierge,No,Xavier Gonzalez,"1101 Ludlow Street, Philadelphia, PA, USA",NaN,NaN,NaN,No,,NaN
1,2024-05-20,06:00:00,14:00:00,NaN,One Park => Valet,No,Eddy Fortuna,"320 Adolphus Avenue, Cliffside Park, NJ, USA",NaN,NaN,NaN,No,None,NaN
2,2024-05-20,07:00:00,15:00:00,NaN,Kenilworth Campus Onyx => Security Rover,No,Ahmed Ibrahim,"2000 Galloping Hill Road, Kenilworth, NJ, USA",NaN,NaN,NaN,No,,NaN
3,2024-05-20,07:00:00,15:00:00,NaN,Botanic => Concierge,No,Nickenson Jean Pierre,"36 Washington Avenue, Carteret, NJ, USA",NaN,NaN,NaN,No,,NaN
4,2024-05-20,07:00:00,15:00:00,NaN,Kenilworth Campus Onyx => Security Front Gate,No,Stephon Holt,"2000 Galloping Hill Road, Kenilworth, NJ, USA",NaN,NaN,NaN,No,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265,2024-05-26,23:30:00,10:00:00,NaN,Edgewater => Concierge,No,Clifton King,"2323 Race Street, Philadelphia, PA, USA",NaN,NaN,NaN,No,,NaN
1266,2024-05-26,23:59:00,08:00:00,NaN,Alexander => Concierge,No,William Lee,"300 Alexander Ct, Philadelphia, PA, USA",NaN,NaN,NaN,No,,NaN
1267,2024-05-26,23:59:00,08:00:00,NaN,The Noble => Concierge,No,Wade Simms,"200 Spring Garden Street, Philadelphia, PA, USA",NaN,NaN,NaN,No,,NaN
1268,2024-05-26,23:59:00,08:00:00,NaN,Ludlow => Concierge,No,Xavier Gonzalez,"1101 Ludlow Street, Philadelphia, PA, USA",NaN,NaN,NaN,No,,NaN


In [145]:
def split_shifts(df):
    split_data = []
    
    for index, row in df.iterrows():
        # Combine Date and Start time into a single datetime object
        start = datetime.combine(row['Date'], row['Start'])
        # Combine Date and End time into a single datetime object
        end = datetime.combine(row['Date'], row['End'])
        
        # Check if the shift ends the next day (overnight shift)
        if end <= start:
            end += timedelta(days=1)  # Adjust end time to the next day
            
            # Split the shift into two parts: before and after midnight
            first_shift_end = datetime.combine(row['Date'], time(23, 59, 59))
            second_shift_start = datetime.combine(row['Date'] + timedelta(days=1), time(0, 0, 0))
            
            # Calculate hours for the first part of the shift (before midnight)
            first_shift_hours = (first_shift_end - start).total_seconds() / 3600 + 1 / 3600
            # Calculate hours for the second part of the shift (after midnight)
            second_shift_hours = (end - second_shift_start).total_seconds() / 3600
            
            # Append the first part of the shift to the split_data list
            split_data.append({
                'Date': row['Date'],
                'Start': row['Start'],
                'End': '11:59:59 PM',
                'Shift title': row['Shift title'],
                'Job': row['Job'],
                'Draft': row['Draft'],
                'Users': row['Users'],
                'Location': row['Location'],
                'Hours': round(first_shift_hours, 2),
                'Holiday': 0  # Initial tagging
            })
            
            # Append the second part of the shift to the split_data list
            split_data.append({
                'Date': row['Date'] + timedelta(days=1),
                'Start': '12:00:00 AM',
                'End': row['End'],
                'Shift title': row['Shift title'],
                'Job': row['Job'],
                'Draft': row['Draft'],
                'Users': row['Users'],
                'Location': row['Location'],
                'Hours': round(second_shift_hours, 2),
                'Holiday': 0  # Initial tagging
            })
        else:
            # Calculate hours if the shift does not span midnight
            hours = (end - start).total_seconds() / 3600
            split_data.append({
                'Date': row['Date'],
                'Start': row['Start'],
                'End': row['End'],
                'Shift title': row['Shift title'],
                'Job': row['Job'],
                'Draft': row['Draft'],
                'Users': row['Users'],
                'Location': row['Location'],
                'Hours': round(hours, 2),
                'Holiday': 0  # Initial tagging
            })
    
    # Return a DataFrame with the split shifts
    return pd.DataFrame(split_data)


In [146]:
df1 = split_shifts(df1)

In [147]:
def holiday_tagger(df):
    # List of holiday dates
    holidays = ['2024-01-01', '2024-05-27', '2024-07-04', '2024-09-02', '2024-12-25', '2024-11-28', '2025-01-01']
    holidays = [datetime.strptime(date, '%Y-%m-%d').date() for date in holidays]
    
    # Convert Date column to datetime.date
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    
    # Tag holidays based on the Date column
    df['Holiday'] = df['Date'].apply(lambda x: 1 if x in holidays else 0)
    
    return df


In [148]:
df1 = holiday_tagger(df1)

In [149]:
def get_info_from_date(df):
       """ Extracts the week number from the date column and adds it to the dataframe"""
       
       df['Week'] = df['Date'].apply(lambda x: x.isocalendar()[1])
       #Ensure that week is an integer # with no decimals and then convert it to a string so that we can filter on it
       
       df['Week'] = df['Week'].apply(lambda x: str(x))
        # get the year from the date column
       df['Year'] = df['Date'].apply(lambda x: x.year)
        # get the month from the date column
       df['Month'] = df['Date'].apply(lambda x: x.month)

        # return the original dataframe, with three new columns, week, year and month
       return df

In [150]:
df1 = get_info_from_date(df1)

In [151]:
df1.head()

,Date,Start,End,Shift title,Job,Draft,Users,Location,Hours,Holiday,Week,Year,Month
0,2024-05-20,01:00:00,08:00:00,NaN,Ludlow => Concierge,No,Xavier Gonzalez,"1101 Ludlow Street, Philadelphia, PA, USA",7.0,0,21,2024,5
1,2024-05-20,06:00:00,14:00:00,NaN,One Park => Valet,No,Eddy Fortuna,"320 Adolphus Avenue, Cliffside Park, NJ, USA",8.0,0,21,2024,5
2,2024-05-20,07:00:00,15:00:00,NaN,Kenilworth Campus Onyx => Security Rover,No,Ahmed Ibrahim,"2000 Galloping Hill Road, Kenilworth, NJ, USA",8.0,0,21,2024,5
3,2024-05-20,07:00:00,15:00:00,NaN,Botanic => Concierge,No,Nickenson Jean Pierre,"36 Washington Avenue, Carteret, NJ, USA",8.0,0,21,2024,5
4,2024-05-20,07:00:00,15:00:00,NaN,Kenilworth Campus Onyx => Security Front Gate,No,Stephon Holt,"2000 Galloping Hill Road, Kenilworth, NJ, USA",8.0,0,21,2024,5


In [152]:
df1.iloc[1:3].Job.str.split('=>').str[0]

1                  One Park 
2    Kenilworth Campus Onyx 
Name: Job, dtype: object

In [153]:
def get_building_and_job(df):
    """ Extracts the building name and the job title from the Job column and adds them to the dataframe by splitting on =>"""
    df['Building'] = df['Job'].str.split('=>').str[0]
    df['Job'] = df['Job'].str.split('=>').str[1]
    return df

In [154]:
df1 = get_building_and_job(df1)

In [155]:
df1.head()

,Date,Start,End,Shift title,Job,Draft,Users,Location,Hours,Holiday,Week,Year,Month,Building
0,2024-05-20,01:00:00,08:00:00,NaN,Concierge,No,Xavier Gonzalez,"1101 Ludlow Street, Philadelphia, PA, USA",7.0,0,21,2024,5,Ludlow
1,2024-05-20,06:00:00,14:00:00,NaN,Valet,No,Eddy Fortuna,"320 Adolphus Avenue, Cliffside Park, NJ, USA",8.0,0,21,2024,5,One Park
2,2024-05-20,07:00:00,15:00:00,NaN,Security Rover,No,Ahmed Ibrahim,"2000 Galloping Hill Road, Kenilworth, NJ, USA",8.0,0,21,2024,5,Kenilworth Campus Onyx
3,2024-05-20,07:00:00,15:00:00,NaN,Concierge,No,Nickenson Jean Pierre,"36 Washington Avenue, Carteret, NJ, USA",8.0,0,21,2024,5,Botanic
4,2024-05-20,07:00:00,15:00:00,NaN,Security Front Gate,No,Stephon Holt,"2000 Galloping Hill Road, Kenilworth, NJ, USA",8.0,0,21,2024,5,Kenilworth Campus Onyx


In [156]:
def create_time_sheet(df):
    """
    Takes a dataframe and categorizes the total hours by each employee into three categories (holiday, regular and overtime hours) and shows the result by month, year, week, building and job to be made

    """

    # Create a new DataFrame with the desired columns
    result = pd.DataFrame(columns=['Users', 'Month', 'Year', 'Holiday Hours', 'Regular Hours', 'Overtime Hours'])
    
    # Group the data by employee and week of year
    grouped = df.groupby(['Users', 'Week', 'Year', 'Month', 'Building', 'Job'])
    
    # Iterate over each group and calculate the hours
    for name, group in grouped:
        holiday_hours = group[group['Holiday'] == 1]['Hours'].sum()
        regular_hours = group[group['Holiday'] == 0]['Hours'].sum()
        overtime_hours = 0
        if regular_hours > 40:
            overtime_hours = regular_hours - 40
            #convert overtime_hours to float

            regular_hours = 40
        
        month = group.loc[:,'Month']
            
       # Create a single-row DataFrame for this group and concatenate it with the result DataFrame
        row = pd.DataFrame({'Users': [name[0]],
                            'Job': [name[4]],
                            'Building': [name[5]],
                            'Week': [name[1]],
                            'Month': [name[3]], 
                            'Year': [name[2]],
                            'Holiday Hours': [holiday_hours], 
                            'Regular Hours': [regular_hours], 
                            'Overtime Hours': [overtime_hours]})
        
        result = pd.concat([result, row], ignore_index=True)
        #convert overtime_hours to float
        result['Overtime Hours'] = result['Overtime Hours'].astype(float)
        #convert week to str
        result['Week'] = result['Week'].astype(str)
        #convert month to str
        result['Month'] = result['Month'].astype(str)
        #convert year to date format
        result['Year'] = result['Year'].astype(str)

    return result

In [157]:
df1 = create_time_sheet(df1)

In [158]:
df1.groupby(["Users"]).agg({"Regular Hours": "sum", "Holiday Hours": "sum", "Overtime Hours": "sum"}).reset_index()

,Users,Regular Hours,Holiday Hours,Overtime Hours
0,Aaron Reaves,32.00,0.0,0.00
1,Abdel Munpen,29.75,0.0,0.00
2,Abdul Khan,8.00,0.0,0.00
3,Abraham Kiki,25.00,7.0,0.00
4,Abraham Reyes,40.00,0.0,8.00
...,...,...,...,...
286,Yesenia Santiago,8.00,0.0,0.00
287,Yosef Zidan,44.00,0.0,0.00
288,Yvonne Amankwah,60.00,0.0,0.75
289,Yvonne Bravo,32.00,0.0,0.00
